In [9]:
# Import 設定
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parents[0]
sys.path.insert(0, PROJECT_DIR.joinpath("src").__str__())  # 不能用 pathlib.PosixPath 傳入

# Step 01: Ingestion

In [10]:
from ingestion.file_loaders.goodnotes import DoclingGoodnotesLoader
loader = DoclingGoodnotesLoader()

loader_results = []
for filepath in ["早安日語-孫寅華part03-page25.pdf",]:
    print(filepath)
    loader_results = loader.load(filepath)

早安日語-孫寅華part03-page25.pdf


2025-08-24 19:51:23,190 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Linux. Use TensorrtExecutionProvider inference by default.
2025-08-24 19:51:23,369 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Linux. Use TensorrtExecutionProvider inference by default.
2025-08-24 19:51:23,395 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Linux. Use TensorrtExecutionProvider inference by default.


In [11]:
loader_results

[LoaderResult(content='\'*\'V+前(在V之前)\n\nIv.s.\n\n<!-- image -->\n\n(二(在V之前)\n\n(在√之作的\n\nIvis.\n\n<!-- image -->\n\n(lesson\'6\'\'3\'\'~\'\'6\'\'4\')\n\nね\n\n\'1\'わたしは\n\n寝る\n\nまえ\n\n前に\n\nふろ\n\nお風呂に\n\nはい\n\nいY\n\n(lesson\'6\'\'3\'\'~\'\'6\'\'4\')\n\nね\n\n型\n\nしは\n\n寝る\n\nまえ\n\n前に\n\nふろ\n\nお風呂に\n\nはい\n\n入ります\n\n<!-- image -->\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\n(lesson\'6\'\'3\'\'~\'\'6\'\'4\')\n\n型\n\nね\n\nしは\n\n寝る\n\nにほ人\n\nふ弖\n\nはい\n\n前二\n\nお風呂に\n\n入ります。\n\n行く\n\nスΩ\n\nこばやし\n\n日本へ\n\n小林さ人です。\n\nめがね\n\n人は\n\nひと\n\n眼鏡を\n\nかけている人は\n\nだれですが。\n\n表現狀\n\nしんぶん\n\n助詞：表場所\n\n新聞を\n\n\'2\'\'=\'\n\nれんしゅう\n\nよ\n\nLY\n\nまえ\n\nいるのは\n\n田中さんです。\n\n(不是助詞)\n\nたなか\n\nひ℃\n\n形式名詞：代替人\n\n\'6\'\n\n在同一句中反覆出现\n\n可以避免同一個名詞\n\nか\n\n中土んです。\n\n同一個名詞\n\n反覆出現\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}

# Step 2: Chunk

In [12]:
from chunking.docling import DoclingChunkProcessor
chunker = DoclingChunkProcessor()

chunks = []
for loader_result in loader_results:
    chunks += chunker.process(doc=loader_result.doc,
                              metadata=loader_result.metadata,
                              )

In [13]:
len(chunks)

61

In [14]:
print(chunks[0]._raw_chunk)

text='\'*\'V+前(在V之前)\nIv.s.\n(二(在V之前)\n(在√之作的\nIvis.\n(lesson\'6\'\'3\'\'~\'\'6\'\'4\')\nね\n\'1\'わたしは\n寝る\nまえ\n前に\nふろ\nお風呂に\nはい\nいY\n(lesson\'6\'\'3\'\'~\'\'6\'\'4\')\nね\n型\nしは\n寝る\nまえ\n前に\nふろ\nお風呂に\nはい\n入ります\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n(lesson\'6\'\'3\'\'~\'\'6\'\'4\')\n型\nね\nしは\n寝る\nにほ人\nふ弖\nはい\n前二\nお風呂に\n入ります。\n行く\nスΩ\nこばやし\n日本へ\n小林さ人です。\nめがね\n人は\nひと\n眼鏡を\nかけている人は\nだれですが。\n表現狀\nしんぶん\n助詞：表場所\n新聞を\n\'2\'\'=\'\nれんしゅう\nよ\nLY\nまえ\nいるのは\n田中さんです。\n(不是助詞)\nたなか\nひ℃\n形式名詞：代替人\n\'6\'\n在同一句中反覆出现\n可以避免同一個名詞\nか\n中土んです。\n同一個名詞\n反覆出現\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\'4\'使其名詞化：+こと\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_n

/tmp/ipykernel_1066308/3635813002.py:1: UserWarning: _raw_chunk 為備份欄位，僅應在 chunking 階段或 debugging 時使用
  print(chunks[0]._raw_chunk)


# Step 3: Embedding

In [15]:
import os
from cache.redis import RedisCacheHandler
from embedding.openai_embed import OpenAIEmbeddingModel


embedder = OpenAIEmbeddingModel(api_key=os.getenv("OPEN_AI_API"),
                                model_name="text-embedding-3-small",
                                memory_cache=RedisCacheHandler(host=os.getenv("MY_REDIS_HOST"),
                                                               port=os.getenv("MY_REDIS_PORT"),
                                                               password=os.getenv("MY_REDIS_PASSWORD"),
                                                               ),
                                )
for idx in range(len(chunks)):
    vector = embedder.encode(chunks[idx].content)
    chunks[idx].embedding = vector

# Step 04: Insert

## Vector-based

In [16]:
from infra.stores.pgvector import PGVectorStore

vec_store = PGVectorStore(host=os.getenv("MY_POSTGRE_HOST"),
                          port=os.getenv("MY_POSTGRE_PORT"),
                          dbname=os.getenv("MY_POSTGRE_DB_NAME"),
                          schema="Japanese-Learning",
                          user=os.getenv("MY_POSTGRE_USERNAME"),
                          password=os.getenv("MY_POSTGRE_PASSWORD"),
                          )

for chunk in chunks:
    vec_store.insert(chunk)

## Lexical-based

In [17]:
from infra.stores.elasticsearch import ElasticsearchBM25Store

lex_store = ElasticsearchBM25Store(host=os.getenv("MY_ELASTIC_HOST"),
                                   port=os.getenv("MY_ELASTIC_PORT"),
                                   index_name="japanese-learning",
                                   username=os.getenv("MY_ELASTIC_USERNAME"),
                                   password=os.getenv("MY_ELASTIC_PASSWORD"),
                                   )

for chunk in chunks:
    lex_store.insert(chunk)

# Step 05: Retrieve

## Vector-based

In [18]:
import os

from cache.redis import RedisCacheHandler
from embedding.openai_embed import OpenAIEmbeddingModel
from infra.stores.pgvector import PGVectorStore

embedder = OpenAIEmbeddingModel(api_key=os.getenv("OPEN_AI_API"),
                                model_name="text-embedding-3-small",
                                memory_cache=RedisCacheHandler(host=os.getenv("MY_REDIS_HOST"),
                                                               port=os.getenv("MY_REDIS_PORT"),
                                                               password=os.getenv("MY_REDIS_PASSWORD"),
                                                               ),
                                )
query_vector = embedder.encode("自動詞與他動詞的判斷邏輯")

vec_store = PGVectorStore(host=os.getenv("MY_POSTGRE_HOST"),
                          port=os.getenv("MY_POSTGRE_PORT"),
                          dbname=os.getenv("MY_POSTGRE_DB_NAME"),
                          schema="Japanese-Learning",
                          user=os.getenv("MY_POSTGRE_USERNAME"),
                          password=os.getenv("MY_POSTGRE_PASSWORD"),
                          )
results = vec_store.search(query_vector)

In [19]:
[_.chunk.content for _ in results]

["此章節介紹「動詞連体形，連体修飾語，之用法(/)\n動詞修飾名詞\n動詞句修飾名詞\n與「比較」文型\n'*'如果詢問他人住哪，幾乎都是使用ます(表狀態持續) Exとこに住人Tいますが わたしは台北に住人でいます\nたんこ\n東\nす\n住む\nたんご\n'*'す\n単語 (lesson'6''6')\n住む\n<\n修飾句主詞一律用が」",
 "此章節介紹\n引述」思考，動词的惯用型態：+V 動詞意向形\n'2'+L\n'*'\nもう\n由す\nも\n申\n単\n他\nに答えます\n(我叫做「林」)",
 "詞性解釋\n他⇒他動詞(似及物動詞).例如':'授舆\n颜理：(他五'>'：五五段变化('6'弥音化动詞)",
 "介绍\n引述」，思考」動詞的惯用型態：+V\n動詞意向形\n'7'+ある\n巽東\nもう",
 '形動詞是並列，關係，可調換\nEx\':\'日本は便利で綺震国です。\n結構同\'5\'\n並列關係，可調換\nは便利て綺震国です。\nにほ人 べんり くに \'6\'.日本はきれいで便利な 国\n結構同\'5\'.\nほ人 べんり くに 本はきれいで便利 国\n\'-\'\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nは人\n本はきれいで便利な\nまんご\n本語の\n習\n(lesson \'2\'\'1\'\'~\'\'2\'\'2\')\n中は\nさんは\nしせつ\nひと\n親切な\n人です\n連体形修飾後方名詞\nじようす\n日本語が(上手です)\n国\nべんきょう\nむずか\nたいへん\n形容詞+形動詞\n勉強は\n難しくて\n大変です。\n由於「難」和「大変」有因果關係，所以不能調換\nすうがく\n若(形動)無法直接形容到主詞\n則會用が做為助詞連接\nす。\n人不能調換\n到主詞\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nTable Part 001\n{\n  "col000":{\n    "

## Lexical-based

In [20]:
import os

from infra.stores.elasticsearch import ElasticsearchBM25Store

lex_store = ElasticsearchBM25Store(host=os.getenv("MY_ELASTIC_HOST"),
                                   port=os.getenv("MY_ELASTIC_PORT"),
                                   index_name="japanese-learning",
                                   username=os.getenv("MY_ELASTIC_USERNAME"),
                                   password=os.getenv("MY_ELASTIC_PASSWORD"),
                                   )
results = lex_store.search("自動詞與他動詞的判斷邏輯")

In [21]:
[_.chunk.content for _ in results]

["つく\n作ったク\n節介紹「動詞連体形，連体修飾語，之用法(/)\n連体修饰句\n動詞修飾名詞\n動詞句修飾名詞\n與比較」文型\n人住哪，幾乎都是使用す(表狀態持續) 住人いますが は台北に住人います\nが\nv.'8'.書<\n修飾\nう\n（他五）建回\n生まれる\n居住\n生まれる\n(ggussə)\n描\nか\nv.s.書く\n他天妻回\nなら\nお\n落とす\n'1'他五'>'弄掉·去\n'~'",
 '形惯用形\nりゅうがく\n名，白\'#\'\'>\'层到\n\'*\'\nけっこん\n白廿结婚\nテストク他サ\'>\'考試\nせつめい\n說明\nテスト(名;他サ\'>\'考試\nせつめい\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\nりゆうがく\n(名·自\'>\'遲到\n\'*\'\nけっこん\nおしゃべ（名·自サ)間談聊天\n(名·自サ)留学\n(名·他升\'>\'預約，預訂\nかいわ\n会話(名自サ\'>\'會話\nガんたん\n(名·形動\'>\'精力充沛;身體結實\n簡単\n蔡草\nまじめ（名形動)認真\n(名形動\'>\'簡單\n台風(名)颱風\n(名)香煙(tabaco)\n一夕一(名)電梯(devator)\nうみ\nガび人\n海(名)海;海洋\n花瓶\nた もの\nガし\nぶっガ\n食べ物(名）食物\n物価(名)物價\nパーティー\nお菓子<名〉點心;糕點\nさゆうにゅう\nガみけ\n\'*\'ガ\n<名)花瓶\n髪の毛(名)頭髮\nえ\n<名)牆壁\n(名)畫;圖畫\n絵\n\'*\'\nキ・うかし+\n結婚\nせつめい\n<名·自\'>\'結婚 テス卜(名·他サ\'>\'考試(tes)\n說明\n<名·他\nおと\nまんが\n电话\n(限物体)\n音\n唸作t\n漫画\n土け\n(名)声音\n\'*\'\n(名)派對;舞會\nしゃない\nらいねん\n来年\n(名)明年\n\'*\'\n(名)車內\nまど\n車內\n(名）窗口\n窓\n(名)漫畫\n酒\n(名）酒，日\nしょうらい\n将来(